# Preparation
<a id=preparation />

In [ ]:
%run "../Functions/6. Time analysis.ipynb"
%run "../Functions/Plot.ipynb"

# April 2018 Playtest

In [ ]:
rmdf1522.columns

In [ ]:
#testUserId1 = 'f4589007-0809-4673-ad50-d985abe4fa65'
#testUserId2 = '653f174e-c23f-4d3a-94f6-d26b53ac7168'
#gform[gform[QUserId] == testUserId2]
#testUserId2 in testUsers

In [ ]:
# rows
phase1Label = 'phase1'
phase2Label = 'phase2'
totalLabel = 'total'

# columns
playersCountLabel = 'players'
respondersCountLabel = 'responders'
respondersTwiceCountLabel = 'twice responders'
prefixFemale = 'f '
prefixMale = 'm '


playtestStatistics = pd.DataFrame(
    data = np.nan,
    index = [phase1Label, phase2Label, totalLabel],
    columns = [playersCountLabel]
)

def setCount(rmdf, rowLabel, columnLabel):
    playtestStatistics.loc[rowLabel, columnLabel] = len(rmdf[QUserId].unique())
    return playtestStatistics.loc[rowLabel, columnLabel]

def setGenderedCount(rmdf, sampledGform, rowLabel, columnLabel, plotPie = False, title = ''):
    femaleResponders = sampledGform[sampledGform[QGender] == "Female"][QUserId]
    maleResponders = sampledGform[sampledGform[QGender] == "Male"][QUserId]

    femaleSample = rmdf[rmdf[QUserId].isin(femaleResponders)]
    maleSample = rmdf[rmdf[QUserId].isin(maleResponders)]

    tCount = setCount(rmdf, rowLabel, columnLabel)
    fCount = setCount(femaleSample, rowLabel, prefixFemale + columnLabel)
    mCount = setCount(maleSample, rowLabel, prefixMale + columnLabel)
    oCount = tCount-fCount-mCount
    
    if plotPie:
        plt.pie([fCount, mCount, oCount], labels=['Female', 'Male', 'Other/Unknown'], autopct='%1.1f%%', shadow=True)
        plt.title(title)
        plt.show()

In [ ]:
def getUniqueProfiles(sampleForm):
    playtestProfiles = sampleForm.loc[:,[QUserId, QAge, QGender]]
    playtestProfiles.drop_duplicates(subset=[QUserId, QAge, QGender],keep='first',inplace=True)
    return playtestProfiles

In [ ]:
def plotAge(sampleForm, title, xlabel = 'Âge', ylabel = 'Nombre'):
    playtestProfiles = getUniqueProfiles(sampleForm)
    
    respondersAge = playtestProfiles[QAge]
    respondersAge = respondersAge.value_counts()
    respondersAge = pd.Series(respondersAge.index.values, index=respondersAge)
    respondersAge.hist(bins=20)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
def plotGender(sampleForm, title = '', labels = ['Femmes', 'Hommes', 'Autres/Inconnu']):
    playtestProfiles = getUniqueProfiles(sampleForm)

    fCount = len(playtestProfiles[playtestProfiles[QGender] == 'Female'])
    mCount = len(playtestProfiles[playtestProfiles[QGender] == 'Male'])
    tCount = len(playtestProfiles)
    oCount = tCount-fCount-mCount

    plt.pie([fCount, mCount, oCount], labels=labels, autopct='%1.1f%%', shadow=True)
    plt.title(title)
    plt.show()
    print("t="+str(tCount))
    print("f="+str(fCount))
    print("m="+str(mCount))
    print("o="+str(oCount))

## Dates

In [ ]:
#rmdf1522.iloc[0]['userTime'],type(rmdf1522.iloc[0]['userTime'])
playtest2018Phase1StartDate = pd.Timestamp('2018-04-10 11:00:00')
playtest2018Phase1EndDate = pd.Timestamp('2018-04-25 19:00:00')
playtest2018Phase2StartDate = pd.Timestamp('2018-04-27 11:00:00')
playtest2018Phase2EndDate = pd.Timestamp('2018-04-28 19:00:00')
playtest2018Phase1StartDate,playtest2018Phase1EndDate,playtest2018Phase2StartDate,playtest2018Phase2EndDate

In [ ]:
# samples based on dates and platform
sampleWhole = getWithoutTest(gform)

rmdfLinuxPlatform = rmdf1522[rmdf1522['customData.platform'] == '"linuxplayer"']
sampleLinux = sampleWhole[sampleWhole[QUserId].isin(rmdfLinuxPlatform[QUserId].unique())]

samplePlaytest = sampleLinux[((sampleLinux[QTimestamp] > playtest2018Phase1StartDate)&(sampleLinux[QTimestamp] < playtest2018Phase2EndDate))]
samplePlaytestPhase1 = samplePlaytest[samplePlaytest[QTimestamp] < playtest2018Phase1EndDate]
samplePlaytestPhase2 = sampleWhole[((sampleWhole[QTimestamp] > playtest2018Phase2StartDate)&(sampleWhole[QTimestamp] < playtest2018Phase2EndDate))]

samplePlaytestPretestPosttest = getPerfectPretestPostestPairs(samplePlaytest)
samplePlaytestPretestPosttestPhase1 = getPerfectPretestPostestPairs(samplePlaytestPhase1)
samplePlaytestPretestPosttestPhase2 = getPerfectPretestPostestPairs(samplePlaytestPhase2)

In [ ]:
rmdfPlaytest = rmdfLinuxPlatform[((rmdfLinuxPlatform['userTime'] > playtest2018Phase1StartDate)&(rmdfLinuxPlatform['userTime'] < playtest2018Phase2EndDate))]
rmdfPlaytest = rmdfPlaytest[rmdfPlaytest[QUserId].isin(samplePlaytest[QUserId].unique())]

rmdfPlaytestPhase1 = rmdfPlaytest[rmdfPlaytest[QUserId].isin(samplePlaytestPhase1[QUserId])]
rmdfPlaytestPhase1 = rmdfPlaytestPhase1[rmdfPlaytestPhase1['userTime'] < playtest2018Phase1EndDate]
rmdfPlaytestPhase2 = rmdfPlaytest[rmdfPlaytest[QUserId].isin(samplePlaytestPhase2[QUserId])]
rmdfPlaytestPhase2 = rmdfPlaytestPhase2[rmdfPlaytestPhase2['userTime'] > playtest2018Phase2StartDate]

rmdfPlaytestPretestPosttest = rmdfPlaytest[rmdfPlaytest[QUserId].isin(samplePlaytestPretestPosttest[QUserId])]
rmdfPlaytestPretestPosttestPhase1 = rmdfPlaytestPhase1[rmdfPlaytestPhase1[QUserId].isin(samplePlaytestPretestPosttestPhase1[QUserId])]
rmdfPlaytestPretestPosttestPhase2 = rmdfPlaytestPhase2[rmdfPlaytestPhase2[QUserId].isin(samplePlaytestPretestPosttestPhase2[QUserId])]

### players

In [ ]:
thresholdGameEventsCount = 10
thresholdTimedelta = pd.Timedelta('0 days 00:05:00.000000')

playedEnoughTime = []
playedEnoughEvents = []

for userId in samplePlaytest[QUserId].unique():
    rows = rmdfPlaytest[rmdfPlaytest[QUserId] == userId]
    if len(rows) > thresholdGameEventsCount:
        playedEnoughEvents.append(userId)
    delta = rows['userTime'].max() - rows['userTime'].min()
    if delta > thresholdTimedelta:
        playedEnoughTime.append(userId)
        
len(samplePlaytest[QUserId].unique()), len(playedEnoughTime), len(playedEnoughEvents)

In [ ]:
setCount(rmdfPlaytestPhase1, phase1Label, playersCountLabel)
setCount(rmdfPlaytestPhase2, phase2Label, playersCountLabel)
setCount(rmdfPlaytest, totalLabel, playersCountLabel)
    
assert(playtestStatistics.loc[totalLabel, playersCountLabel] == \
    playtestStatistics.loc[phase1Label, playersCountLabel]\
    + playtestStatistics.loc[phase2Label, playersCountLabel])

playtestStatistics

### all playtest responders

In [ ]:
plotGender(samplePlaytest,'Sexe des participants, phase 1')

In [ ]:
plotGender(samplePlaytestPhase2,'Sexe des participants, phase 2')

In [ ]:
len(playtestProfiles[QAge])

In [ ]:
playtestProfiles[QAge].value_counts()

In [ ]:
playtestProfiles = getUniqueProfiles(samplePlaytest)
    
respondersAge = playtestProfiles[QAge]
respondersAge = respondersAge.value_counts()
respondersAge = pd.Series(respondersAge.index.values, index=respondersAge)
respondersAge.hist()#bins=20)
plt.xlabel('xlabel')
plt.ylabel('ylabel')
plt.title('title')
plt.show()

In [ ]:
plotAge(samplePlaytest, "Âge des participants, phase 1")

In [ ]:
plotAge(samplePlaytestPhase2, "Âge des participants, phase 2")

In [ ]:
setGenderedCount(rmdfPlaytestPretestPosttestPhase1, samplePlaytestPhase1, phase1Label, respondersCountLabel)
setGenderedCount(rmdfPlaytestPretestPosttestPhase2, samplePlaytestPhase2,  phase2Label, respondersCountLabel)
setGenderedCount(rmdfPlaytestPretestPosttest, samplePlaytest,  totalLabel, respondersCountLabel, plotPie = True, title = 'Gender of participants (all)')

assert(playtestStatistics.loc[totalLabel, respondersCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondersCountLabel]\
      + playtestStatistics.loc[phase2Label, respondersCountLabel])

playtestStatistics

In [ ]:
# answers per userId
# => gform.indexes per userId
indicesPerUserId = gform[QUserId].value_counts()
#indicesPerUserId

### twice responders

In [ ]:
twiceResponders = getPerfectPretestPostestPairs(gform)
playtestTwiceResponders = twiceResponders[twiceResponders[QUserId].isin(samplePlaytest[QUserId])]

uniquePlaytestPhase1TwiceResponders = samplePlaytestPhase1[samplePlaytestPhase1[QUserId].isin(playtestTwiceResponders[QUserId])]
uniquePlaytestPhase2TwiceResponders = samplePlaytestPhase2[samplePlaytestPhase2[QUserId].isin(playtestTwiceResponders[QUserId])]
uniquePlaytestTwiceResponders = samplePlaytest[samplePlaytest[QUserId].isin(playtestTwiceResponders[QUserId])]

setGenderedCount(uniquePlaytestPhase1TwiceResponders, playtestTwiceResponders, phase1Label, respondersTwiceCountLabel)
setGenderedCount(uniquePlaytestPhase2TwiceResponders, playtestTwiceResponders,  phase2Label, respondersTwiceCountLabel)
setGenderedCount(uniquePlaytestTwiceResponders, playtestTwiceResponders,  totalLabel, respondersTwiceCountLabel, plotPie = True, title = 'Gender of participants (pretest-posttest)')

assert(playtestStatistics.loc[totalLabel, respondersTwiceCountLabel] ==\
       playtestStatistics.loc[phase1Label, respondersTwiceCountLabel]\
      + playtestStatistics.loc[phase2Label, respondersTwiceCountLabel])

playtestStatistics

In [ ]:
plotAge(allPlaytestResponders, 'Age repartition of all participants')

In [ ]:
plotAge(playtestTwiceResponders, 'Age repartition of pretest-posttest participants')